In [1]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from alexnet import *
from classes import class_names

In [2]:
# Misc
LOG_DIR = './log/'

# Training Parameters
LEARNING_RATE = 0.001
NUM_EPOCHS = 1
NUM_STEPS = 500
BATCH_SIZE = 128
DISPLAY_STEP = 10

# Network Parameters
NUM_CLASSES = 1000
KEEP_RATE = 0.75

In [3]:
def read_image (path):
    # Read Image
    img_file = tf.read_file(path)
    img_decoded = tf.image.decode_jpeg(img_file, channels=3)
    img_resized = tf.image.resize_images(img_decoded, [227, 227])
    
    return img_resized
    

In [4]:
model_graph = tf.Graph()
with model_graph.as_default():
    
    data = tf.data.Dataset.from_tensor_slices(np.array(['./image/llama.jpeg']))
    data = data.map(read_image).batch(1)
    iterator = data.make_one_shot_iterator()
    next_el = iterator.get_next()
    
    #images = tf.placeholder(tf.float32, [None, 227, 227, 3])
    #labels = tf.placeholder(tf.float32, [None, NUM_CLASSES])
    keep_prob = tf.placeholder(tf.float32)
    
    # Build Model
    alexnet = My_AlexNet(x=next_el, keep_rate=keep_prob, 
                         num_classes=NUM_CLASSES, skip_layers=[],
                         weights_path='../../bvlc_alexnet.npy')
    alexnet.build()
    
    # Logits and Predictions
    logits = alexnet.logits
    prediction = {'classes': tf.argmax(logits, axis=1), 
                  'prob': tf.nn.softmax(logits, name='prob')}
    
    init = tf.global_variables_initializer()

In [10]:
writer = tf.summary.FileWriter(os.path.join(LOG_DIR, 'model_1'), graph=model_graph)

In [65]:
with tf.Session(graph=model_graph) as sess:
    
    sess.run(init)
    
    alexnet.load_weights(session=sess, encoding='latin1')
    
    pred = sess.run(prediction, feed_dict={keep_prob: 1.0})
    
    print(class_names[pred['classes'][0]])
    print(pred['prob'])

ValueError: Dimension 1 in both shapes must be equal, but are 2 and 1000. Shapes are [4096,2] and [4096,1000]. for 'fc8_1/Assign' (op: 'Assign') with input shapes: [4096,2], [4096,1000].

In [1]:
import tensorflow as tf
import numpy as np
from alexnet import *

In [2]:
weights_dict = np.load('../../bvlc_alexnet.npy', encoding='latin1').item()

In [3]:
images = tf.placeholder(tf.float32, [None, 227, 227, 3])
labels = tf.placeholder(tf.float32, [None, 1000])
keep_prob = tf.placeholder(tf.float32)

# Build Model
alexnet = My_AlexNet(x=images, keep_rate=keep_prob, 
                     num_classes=1000)
alexnet.build()

init = tf.global_variables_initializer()

In [4]:
sess = tf.Session()

In [5]:
sess.run(init)

In [7]:
op_name = 'fc6'

with tf.variable_scope(op_name, reuse=tf.AUTO_REUSE):

    var = tf.get_variable('weights', shape=weights_dict[op_name][0].shape)

    sess.run(var.assign(weights_dict[op_name][0]))
